In [0]:
import re

from pyspark.sql import Row

# Il s'agit de l'expression régulière spécifique à l'analyse des logs Apache, qui peut être modifiée en fonction de différents formats de journaux selon les besoins
# Exemple d'une ligne de log Apache:
#                              127.0.0.1 - - [21/Jul/2014:9:55:27 -0800] "GET /home.html HTTP/1.1" 200 2048
#                              1:IP  2:client 3:user 4:date time           5:method 6:req 7:proto   8:respcode 9:size
APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\d+)'

# La fonction ci-dessous est modélisée spécifiquement au modèle Apache Access Logs, qui peut être modifié selon les besoins en différents formats de journaux
# Renvoie un dictionnaire contenant les parties du journal d'accès Apache.
def parse_apache_log_line(logline):
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
    if match is None:
        raise Error("Invalid logline: %s" % logline)
    return Row(
        ip_address    = match.group(1),
        client_identd = match.group(2),
        user_id       = match.group(3),
        date = (match.group(4)[:-6]).split(":", 1)[0],
        time = (match.group(4)[:-6]).split(":", 1)[1],
        method        = match.group(5),
        endpoint      = match.group(6),
        protocol      = match.group(7),
        response_code = int(match.group(8)),
        content_size  = int(match.group(9))
    )

In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import sys


#fichier d'entrée 
logFile = '/FileStore/tables/apache_access.log'

# .cache() - Conserve le RDD en mémoire, qui sera réutilisé
access_logs = (sc.textFile(logFile)
               .map(parse_apache_log_line)
               .cache())
access_logs.count()

Out[2]: 1406

In [0]:
schema_access_logs = sqlContext.createDataFrame(access_logs)
#Crée une table sur laquelle des requêtes de type SQL peuvent être déclenchées pour analyse
schema_access_logs.registerTempTable("logs")

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [0]:
# la taille du trafic par adresse (page ou endpoint)
topEndpointsMaxSize = (sqlContext
                .sql("SELECT endpoint,content_size/1024 FROM logs ORDER BY content_size DESC LIMIT 10")
                .rdd.map(lambda row: (row[0], row[1]))
                .collect())

In [0]:
display(topEndpointsMaxSize)

_1,_2
/twiki/bin/rdiff/TWiki/TWikiHistory,135.5361328125
/twiki/bin/rdiff/Main/WebChanges,111.5634765625
/twiki/bin/rdiff/Main/WebChanges?rev1=1.2&rev2=1.1,111.54296875
/twiki/bin/rdiff/TWiki/WebChanges,111.4912109375
/twiki/bin/rdiff/TWiki/WebHome,105.626953125
/twiki/bin/search/TWiki/?scope=topic®ex=on&bookview=on&search=.*,99.9990234375
/twiki/bin/view/TWiki/WebIndex,99.759765625
/twiki/bin/rdiff/TWiki/TWikiTopics,99.0673828125
/twiki/bin/search/TWiki/?search=\\.*&scope=topic&order=modified&reverse=on®ex=on&nosearch=on&limit=200,98.9052734375
/twiki/bin/rdiff/TWiki/TWikiSite,70.2548828125


In [0]:
# Afficher le nombre de fois qu'un code de réponee a été déclenché 
responseCodeToCount = (sqlContext
                       .sql("SELECT response_code, COUNT(*) AS theCount FROM logs GROUP BY response_code")
                       .rdd.map(lambda row: (row[0], row[1]))
                       .collect())
type(responseCodeToCount)

Out[6]: list

In [0]:
display(responseCodeToCount)

_1,_2
404,5
200,1272
401,123
302,6


In [0]:
# le nombre d'occurence de chaque adresse IP qui a navigué sur le site (seules les adresse avec un total supérieur à 10 seront affichées)
frequentIpAddressesHits = (sqlContext
               .sql("SELECT ip_address, COUNT(*) AS total FROM logs GROUP BY ip_address HAVING total > 10")
               .rdd.map(lambda row: (row[0], row[1])))

In [0]:
frequentIpAddressesHits

Out[9]: PythonRDD[85] at RDD at PythonRDD.scala:58

In [0]:
frequentIpAddressesHits=sqlContext.createDataFrame(frequentIpAddressesHits)

In [0]:
display(frequentIpAddressesHits)


_1,_2
128.227.88.79,12
p213.54.168.132.tisdip.tiscali.de,12
prxint-sxb3.e-i.net,14
10.0.0.153,188
market-mail.panduit.com,29
lhr003a.dhl.com,13
212.92.37.62,14
64.242.88.10,452
cr020r01-3.sac.overture.com,44
216-160-111-121.tukw.qwest.net,12


In [0]:
# la taille du trafic par date 
trafficWithTime = (sqlContext
                       .sql("SELECT date, content_size/1024 FROM logs")
                       .rdd.map(lambda row: (row[0], row[1])))
               

In [0]:
trafficWithTime=sqlContext.createDataFrame(trafficWithTime)
trafficWithTime.count()

Out[13]: 1406

In [0]:
display(trafficWithTime)

_1,_2
07/Mar/2004,12.544921875
07/Mar/2004,4.4169921875
07/Mar/2004,6.1435546875
07/Mar/2004,7.1796875
07/Mar/2004,5.1298828125
07/Mar/2004,11.115234375
07/Mar/2004,4.80859375
07/Mar/2004,12.5498046875
07/Mar/2004,12.5498046875
07/Mar/2004,3.64453125


### Deux visualisations au choix

In [0]:
# la taille du trafic par protocol 
trafficWithprotocol = (sqlContext
                       .sql("SELECT protocol, content_size/1024 FROM logs")
                       .rdd.map(lambda row: (row[0], row[1])))

In [0]:
trafficWithprotocol=sqlContext.createDataFrame(trafficWithprotocol)
trafficWithprotocol.count()

Out[33]: 1406

In [0]:
display(trafficWithprotocol)

_1,_2
HTTP/1.1,12.544921875
HTTP/1.1,4.4169921875
HTTP/1.1,6.1435546875
HTTP/1.1,7.1796875
HTTP/1.1,5.1298828125
HTTP/1.1,11.115234375
HTTP/1.1,4.80859375
HTTP/1.1,12.5498046875
HTTP/1.1,12.5498046875
HTTP/1.1,3.64453125


In [0]:
# la taille du trafic par method
trafficWithmethod = (sqlContext
                       .sql("SELECT method, content_size/1024 FROM logs")
                       .rdd.map(lambda row: (row[0], row[1])))

In [0]:
trafficWithmethod =sqlContext.createDataFrame(trafficWithmethod )
trafficWithmethod.count()

Out[38]: 1406

In [0]:
display(trafficWithmethod)

_1,_2
GET,12.544921875
GET,4.4169921875
GET,6.1435546875
GET,7.1796875
GET,5.1298828125
GET,11.115234375
GET,4.80859375
GET,12.5498046875
GET,12.5498046875
GET,3.64453125
